# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [107]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql.cursors
from pymysql import IntegrityError

import warnings
warnings.filterwarnings("ignore") #supress warnings

## Import Data

In [108]:
# API endpoint
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# requesting data from API
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()

    # data update check
    last_modified_header = response.headers.get('Last-Modified')

    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to dataframe in chunks
    chunk_size = 100  # define size of chunk
    hospital_dfs = []  # create list to store df 

    for chunk_start in range(0, len(data), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(data))
        chunk_data = data[chunk_start:chunk_end]
        hospital_df = pd.DataFrame(chunk_data)
        hospital_dfs.append(hospital_df)

    # concate all df chunks into one
    hospital_df = pd.concat(hospital_dfs, ignore_index=True)

    # print df 
    print(hospital_df)
    
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
300          301              46  941196203   
301          302             209  952082686   
302          303             210  952294234   
303          304              46  941196203   
304          305             151  940535360   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
300           Mercy Medical Center Merced   
301          Ridgecrest Regional Hospital   
302          Adventist Health Bakersfield   
303               St Marys Medical Center   
304        Adventist Medical Center-Selma   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [109]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
300,301,46,941196203,Mercy Medical Center Merced,Mercy Medical Center Merced,333 Mercy Avenue,Merced,CA,95340,06047,186,N,Y,N,Y,050444,Merced County,100-299 beds,"November 20, 2023"
301,302,209,952082686,Ridgecrest Regional Hospital,Ridgecrest Regional Hospital,1081 North China Lake Blvd,Ridgecrest,CA,93555,06029,80,N,Y,N,N,050448,Kern County,<100 beds,"November 20, 2023"
302,303,210,952294234,Adventist Health Bakersfield,Adventist Health Bakersfield,2615 Chester Avenue,Bakersfield,CA,93303,06029,255,N,Y,N,N,050455,Kern County,100-299 beds,"November 20, 2023"
303,304,46,941196203,St Marys Medical Center,St Marys Medical Center,450 Stanyan Street,San Francisco,CA,94117,06075,339,N,Y,N,Y,050457,San Francisco County,>299 beds,"November 20, 2023"
304,305,151,940535360,Adventist Medical Center-Selma,,1141 Rose Ave,Selma,CA,93662,06019,57,N,Y,N,Y,050470,Fresno County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [110]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
3067,3068,2291,741282696,Grimes St Joseph Health Center,Grimes St Joseph Health Center,210 Judson,Navasota,TX,77868,48185,25,Y,N,N,N,451322,Grimes County,<100 beds,"November 20, 2023"
1666,1667,1297,870741588,Simpson General Hospital Cah,Simpson General Hospital Cah,1842 Simpson Hwy 149,Mendenhall,MS,39114,28127,35,N,Y,N,N,251317,Simpson County,<100 beds,"November 20, 2023"
607,608,415,581954432,Northside Hospital-Cherokee Inc,Northside Hospital-Cherokee Inc,201 Hospital Road,Canton,GA,30115,13057,84,N,Y,N,N,110008,Cherokee County,<100 beds,"November 20, 2023"
1882,1883,1470,020232673,Elliot Hospital,Elliot Hospital,1 Elliot Way,Manchester,NH,03103,33011,296,N,Y,N,N,300012,Hillsborough County,100-299 beds,"November 20, 2023"
1760,1761,1363,810231787,Northern Montana Hospital,Northern Montana Hospital,30 13th Street,Havre,MT,59501,30041,49,N,N,N,N,270032,Hill County,<100 beds,"November 20, 2023"
642,643,454,580566213,Piedmont Hospital Inc,Piedmont Hospital Inc,1968 Peachtree Rd Nw,Atlanta,GA,30309,13121,458,N,Y,N,Y,110083,Fulton County,>299 beds,"November 20, 2023"
1008,1009,731,421418847,Genesis Medical Center - Dewitt,Genesis Medical Center - Dewitt,1118 Eleventh St,Dewitt,IA,52742,19045,90,N,N,N,N,161313,Clinton County,<100 beds,"November 20, 2023"
392,393,246,952224265,Santa Ynez Valley Cottage Hospital,Santa Ynez Valley Cottage Hospital,2050 Viborg Road,Solvang,CA,93463,06083,11,N,Y,N,N,051331,Santa Barbara County,<100 beds,"November 20, 2023"
2986,2987,2321,741287016,Methodist Hospital,Methodist Hospital,7700 Floyd Curl Drive,San Antonio,TX,78229,48029,2071,N,Y,N,Y,450388,Bexar County,>299 beds,"November 20, 2023"
1858,1859,1453,470379834,Beatrice Community Hospital,Beatrice Community Hospital,4800 Hospital Parkway,Beatrice,NE,68310,31067,25,N,N,N,N,281364,Gage County,<100 beds,"November 20, 2023"


## Data Inspection

In [111]:
hospital_df.shape 

(3191, 19)

In [112]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3191 entries, 300 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3191 non-null   object
 1   hospital_org_id             3191 non-null   object
 2   ein                         3191 non-null   object
 3   name                        3191 non-null   object
 4   name_cr                     3191 non-null   object
 5   street_address              3191 non-null   object
 6   city                        3191 non-null   object
 7   state                       3191 non-null   object
 8   zip_code                    3191 non-null   object
 9   fips_state_and_county_code  3191 non-null   object
 10  hospital_bed_count          3191 non-null   object
 11  chrch_affl_f                3191 non-null   object
 12  urban_location_f            3191 non-null   object
 13  children_hospital_f         3191 non-null   ob

In [113]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# convert updated_dt to date type
hospital_df['updated_dt'] = hospital_df['updated_dt'].apply(lambda x: datetime.strptime(x, '%B %d, %Y').strftime('%Y-%m-%d'))

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3191 entries, 300 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3191 non-null   object
 1   hospital_org_id             3191 non-null   object
 2   ein                         3191 non-null   object
 3   name                        3191 non-null   object
 4   name_cr                     3191 non-null   object
 5   street_address              3191 non-null   object
 6   city                        3191 non-null   object
 7   state                       3191 non-null   object
 8   zip_code                    3191 non-null   object
 9   fips_state_and_county_code  3191 non-null   object
 10  hospital_bed_count          3191 non-null   int64 
 11  chrch_affl_f                3191 non-null   int64 
 12  urban_location_f            3191 non-null   int64 
 13  children_hospital_f         3191 non-null   in

In [114]:
hospital_df.nunique() #unique variables 

hospital_id                   3191
hospital_org_id               2210
ein                           2210
name                          3053
name_cr                       2812
street_address                3146
city                          2035
state                           47
zip_code                      2933
fips_state_and_county_code    1649
hospital_bed_count             678
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3191
county                        1133
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [115]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        0
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [116]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [117]:
# connect to database

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password, 
                       db= "ads507final", 
                       ssl={"fake_flag_to_enable_tls": True})

In [118]:
# Hospital Information Table

# create unique ID for address using the max value from table
cursor = conn.cursor()
query = "SELECT MAX(Address_id) FROM address_location"
cursor.execute(query)


max_value = cursor.fetchone()[0]
hospital_df['address_id'] = range(max_value + 1, max_value + len(hospital_df) + 1)

#create unique ID for hospital capacity
query = "SELECT MAX(capacity_id) FROM hospital_capacity"
cursor.execute(query)


max_value = cursor.fetchone()[0]
hospital_df['capacity_id'] = range(max_value + 1, max_value + len(hospital_df) + 1)

# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
300,301,46,Mercy Medical Center Merced,941196203,50444,301,301,2023-11-20
301,302,209,Ridgecrest Regional Hospital,952082686,50448,302,302,2023-11-20
302,303,210,Adventist Health Bakersfield,952294234,50455,303,303,2023-11-20
303,304,46,St Marys Medical Center,941196203,50457,304,304,2023-11-20
304,305,151,Adventist Medical Center-Selma,940535360,50470,305,305,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,3487,3487,2023-11-20
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,3488,3488,2023-11-20
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,3489,3489,2023-11-20
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,3490,3490,2023-11-20


In [119]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
300,301,0,2023-11-20
301,302,0,2023-11-20
302,303,0,2023-11-20
303,304,0,2023-11-20
304,305,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [120]:
# Hospital Capacity Table extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
300,301,100-299 beds,186,2023-11-20
301,302,<100 beds,80,2023-11-20
302,303,100-299 beds,255,2023-11-20
303,304,>299 beds,339,2023-11-20
304,305,<100 beds,57,2023-11-20
...,...,...,...,...
3486,3487,<100 beds,16,2023-11-20
3487,3488,<100 beds,7,2023-11-20
3488,3489,<100 beds,63,2023-11-20
3489,3490,<100 beds,46,2023-11-20


In [121]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
300,301,0,2023-11-20
301,302,0,2023-11-20
302,303,0,2023-11-20
303,304,0,2023-11-20
304,305,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [122]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
300,301,1,2023-11-20
301,302,0,2023-11-20
302,303,0,2023-11-20
303,304,1,2023-11-20
304,305,1,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [123]:
# Urban Table
urban = hospital_df[['urban_location_f', 'hospital_id', 'updated_dt']]
urban

,urban_location_f,hospital_id,updated_dt
300,1,301,2023-11-20
301,1,302,2023-11-20
302,1,303,2023-11-20
303,1,304,2023-11-20
304,1,305,2023-11-20
...,...,...,...
3486,1,3487,2023-11-20
3487,1,3488,2023-11-20
3488,1,3489,2023-11-20
3489,1,3490,2023-11-20


In [124]:
# Address/Location Table
Address = hospital_df[['address_id', 'street_address','city','state','zip_code','fips_state_and_county_code','county','updated_dt']]
Address

,address_id,street_address,city,state,zip_code,fips_state_and_county_code,county,updated_dt
300,301,333 Mercy Avenue,Merced,CA,95340,06047,Merced County,2023-11-20
301,302,1081 North China Lake Blvd,Ridgecrest,CA,93555,06029,Kern County,2023-11-20
302,303,2615 Chester Avenue,Bakersfield,CA,93303,06029,Kern County,2023-11-20
303,304,450 Stanyan Street,San Francisco,CA,94117,06075,San Francisco County,2023-11-20
304,305,1141 Rose Ave,Selma,CA,93662,06019,Fresno County,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,5245 W Us 290,Austin,TX,78735,48453,Travis County,2023-11-20
3487,3488,630 Highway 71 W,Bastrop,TX,78602,48021,Bastrop County,2023-11-20
3488,3489,12400 N Dallas Parkway,Frisco,TX,75033,48121,Denton County,2023-11-20
3489,3490,1201 E Highway 287,Midlothian,TX,76065,48139,Ellis County,2023-11-20


In [125]:
tableNames=pd.read_sql_query("""SHOW TABLES""", conn)

# print table names 
tableNames

,Tables_in_ads507final
0,address_location
1,children_hospital
2,church_affiliation
3,hospital
4,hospital_capacity
5,teaching
6,urban


In [126]:
# Address/Location table - convert df to list of tuples
address_data = [tuple(x) for x in Address.values]

# SQL query - insert data into address table
sql_insert_address_data = """
INSERT INTO address_location (Address_id, Street_address, City, State, Zip_code, fips_state_and_county_code, county, Updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# execute query 
if address_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_address_data, address_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into address_location table.")

In [127]:
# hospital table - convert df to list of tuples
hospital_data = [tuple(x) for x in hospital.values]

# SQL query - insert data into hospital table
sql_insert_hospital = """
INSERT INTO hospital (hospital_id, hospital_org_id, name, ein, medicare_provider_number, Address_id, capacity_id, updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital, hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital table.")

In [128]:
# children hospital table - convert df to list of tuples
children_hospital_data = [tuple(x) for x in children_hospital.values]

# SQL query - insert data into children_hospital table
sql_insert_children_hospital = """
INSERT INTO children_hospital(hospital_id, children_hospital_f, updated_dt)
VALUES (%s, %s, %s)
"""

# check if insertion is needed
if children_hospital_data:
    # execute query
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_children_hospital, children_hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into children_hospital table.")

In [129]:
# hospital_capacity table - convert df to list of tuples
hospital_capacity_data = [tuple(x) for x in hospital_capacity.values]

# SQL query - insert data into hospital_capacity table
sql_insert_hospital_capacity = """
INSERT INTO hospital_capacity (capacity_id, hospital_bed_size, hospital_bed_count, updated_dt)
VALUES (%s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_capacity_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital_capacity, hospital_capacity_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital_capacity table.")

In [130]:
# teaching table - convert df to list of tuples
teaching_data = [tuple(x) for x in teaching.values]

# SQL query - insert data into children_hospital table
sql_insert_teaching = """
INSERT INTO teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if teaching_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_teaching, teaching_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into teaching table.")

In [131]:
# church_affiliation table - convert df to list of tuples
church_affiliation_data = [tuple(x) for x in church_affiliation.values]

# SQL query - insert data into church_affiliation table
sql_insert_church_affiliation = """
INSERT INTO church_affiliation (hospital_id, chrch_affl_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if church_affiliation_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_church_affiliation, church_affiliation_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into church_affiliation table.")

In [132]:
# Urban table - convert df to list of tuples
urban_data = [tuple(x) for x in urban.values]

# SQL query - insert data into address table
sql_insert_urban_data = """
INSERT INTO urban (urban_location_f, hospital_id, Updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if urban_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_urban_data, urban_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into urban table.")

In [133]:
# Create a cursor
cursor = conn.cursor()

# Execute SQL queries to get table names
cursor.execute("SHOW TABLES")

# Fetch all the tables
tables = cursor.fetchall()

for table in tables:
        table_name = table[0]

        # Execute SQL query to fetch sample data from each table
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")

        # Fetch and print sample data
        sample_data = cursor.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)

Sample data from table 'address_location':
(1, '702 Main Street', 'Opp', 'AL', 36462, '01039', 'Covington County', datetime.date(2023, 11, 20))
(2, '50 Medical Park Drive East', 'Birmingham', 'AL', 35235, '01073', 'Jefferson County', datetime.date(2023, 11, 20))
(3, '1000 First Street North', 'Alabaster', 'AL', 35007, '01117', 'Shelby County', datetime.date(2023, 11, 20))
(4, '1720 University Boulevard', 'Birmingham', 'AL', 35233, '01073', 'Jefferson County', datetime.date(2023, 11, 20))
(5, '400 Northwood Drive', 'Centre', 'AL', 35960, '01019', 'Cherokee County', datetime.date(2023, 11, 20))
Sample data from table 'children_hospital':
(46288, 1, b'0', datetime.date(2023, 11, 20))
(46289, 2, b'0', datetime.date(2023, 11, 20))
(46290, 3, b'0', datetime.date(2023, 11, 20))
(46291, 4, b'0', datetime.date(2023, 11, 20))
(46292, 5, b'0', datetime.date(2023, 11, 20))
Sample data from table 'church_affiliation':
(38402, 1, b'0', datetime.date(2023, 11, 20))
(38403, 2, b'0', datetime.date(2023